For a more detailed guide refer to `tensorflow` or `pytorch` example or to the documentation on https://clipper.ai.
This example uses `matplotlib`, in case you cannot use it, please comment out respective 2 cells - 7 & 8

In [113]:
# Imports related to ML
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from joblib import dump, load
import numpy as np

In [104]:
# Imports related to Clipper
from clipper_admin.deployers import python as python_deployer
from clipper_admin.deployers import pytorch as pytorch_deployer

from clipper_admin import ClipperConnection, DockerContainerManager, NomadContainerManager, ConsulDNS

In [123]:
CLIPPER_APP_1_NAME='test-app-1'
CLIPPER_APP_2_NAME='test-app-2'

container_registry = 'rg.fr-par.scw.cloud/hyperplan'

def get_clipper_connection():
    nomad_ip_addr = '10.65.30.43'
    dns = ConsulDNS() # We use Consul for DNS resolution
    container_manager = NomadContainerManager(
        nomad_ip=nomad_ip_addr,
        dns=dns
    )
    clipper_conn = ClipperConnection(container_manager)
    clipper_conn.connect()
    return clipper_conn

def train_sklearn(X, y):
    clf = svm.SVC(gamma=0.001, C=100.)
    clf.fit(X, y)
    return clf


def deploy_all():
    conn = get_connection()
    conn.connect()
        
def create_clipper_application(name):
    clipper_connection.register_application(
        name=name, 
        input_type="doubles", 
        default_output="-1.0", 
        slo_micros=10000000
    )
    
def deploy_sklearn(clipper_connection, clf, name, version):
    python_deployer.deploy_python_closure(clipper_connection, name=name, 
        version=version, 
        input_type="doubles", 
        func=clf.predict,
        pkgs_to_install=['scikit-learn'],
        registry=container_registry
    )
    
    
def pytorch_func(model, inputs):
    return [model(x).tolist() for x in torch.Tensor(inputs)]

def deploy_pytorch(clipper_connection, model, name, version):
    pytorch_deployer.deploy_pytorch_model(clipper_connection, name=name, 
        version=version, 
        input_type="doubles", 
        func=pytorch_func,
        pytorch_model=model,
        registry=container_registry
    )

In [99]:
# Load a dataset
diabetes = datasets.load_diabetes() # load data

In [100]:
diabetes.data.shape # feature matrix shape

(442, 10)

# Train Scikit Learn Model

In [101]:
# Seperate train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.33222203269065176

In [7]:
model.predict(X_test) # Predict unkown data

array([238.47145247, 248.93170646, 164.05404165, 120.30794355,
       187.42422054, 259.04865002, 113.55556372, 188.07597044,
       149.49663441, 236.01099949, 172.24629506, 178.88073764,
       109.15751983,  92.13508975, 243.33042043,  87.356971  ,
       155.72606406,  66.99073989, 100.42610442, 218.09422877,
       196.66287912, 161.29832968, 161.70779605, 156.52520454,
       197.88796516, 167.57984206, 120.74478913,  84.83879727,
       192.03728687, 160.60687024, 175.17178362,  84.22833237,
       145.7995542 , 145.97333493, 140.96488953, 197.00421108,
       165.94322494, 190.65906468, 128.22520508, 206.41941223,
        84.35851196, 164.0256504 , 144.1056776 , 184.68355549,
       177.80238966,  74.32855231, 143.3660286 , 138.67726085,
       120.81146113, 234.34252077, 161.94390244,  74.5455476 ,
       154.71905074, 156.78884927, 237.42227096, 174.23053048,
       190.88212635, 118.98373473, 132.20418974, 168.52674824,
       214.74245466, 171.42364091, 157.37409906, 108.86

# Train PyTorch

In [115]:
import torch

# We train a simple Neural Network

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1, 10, 5, 1

pytorch_model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_out)
)

loss_fn = torch.nn.MSELoss()

learning_rate = 1e-1
for t in range(500):
    y_pred = pytorch_model(torch.Tensor(X_train))
    loss = loss_fn(y_pred, torch.Tensor(y_train))
    if t % 100 == 99:
        print(t, loss.item())
    pytorch_model.zero_grad()

    loss.backward()
    with torch.no_grad():
        for param in pytorch_model.parameters():
            param -= learning_rate * param.grad

/Users/antoine/miniconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([353])) that is different to the input size (torch.Size([353, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


99 6130.69873046875
199 6130.69873046875
299 6130.69873046875
399 6130.69873046875
499 6130.6982421875


In [116]:
pytorch_model(torch.Tensor(X_test)) # Predict unkown data

tensor([[151.5620],
        [151.5653],
        [151.6129],
        [151.6360],
        [151.5405],
        [151.5629],
        [151.6113],
        [151.5931],
        [151.5957],
        [151.5638],
        [151.6361],
        [151.6373],
        [151.6411],
        [151.6518],
        [151.6001],
        [151.5846],
        [151.5803],
        [151.6217],
        [151.5939],
        [151.5944],
        [151.5695],
        [151.5972],
        [151.6085],
        [151.5949],
        [151.5914],
        [151.5977],
        [151.5436],
        [151.6143],
        [151.5639],
        [151.6114],
        [151.5707],
        [151.6295],
        [151.5873],
        [151.5984],
        [151.6254],
        [151.5961],
        [151.6233],
        [151.6345],
        [151.6017],
        [151.5857],
        [151.6507],
        [151.5903],
        [151.6137],
        [151.6220],
        [151.6250],
        [151.6567],
        [151.5832],
        [151.5680],
        [151.6223],
        [151.5500],


In [102]:
# Retrieve a connection to Clipper
clipper_connection = get_clipper_connection()

19-10-22:11:02:13 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at 


In [106]:
# Deploy Scikit-Learn
create_clipper_application(CLIPPER_APP_1_NAME)
sklearn_model_name = 'scikit-learn-model'
# Deploy to production but do not serve
deploy_sklearn(clipper_connection, model, name=sklearn_model_name, version=1)
# Link and serve !
#clipper_connection.link_model_to_app(app_name=CLIPPER_APP_1_NAME, model_name=sklearn_model_name)

19-10-22:11:07:48 INFO     [clipper_admin.py:236] [default-cluster] Application test-app-1 was successfully registered
19-10-22:11:07:48 INFO     [deployer_utils.py:41] Saving function to /var/folders/k0/gp0mgjf54ml_rr6bx2c_h7bw0000gn/T/tmpny6my18lclipper
19-10-22:11:07:48 INFO     [deployer_utils.py:51] Serialized and supplied predict function
19-10-22:11:07:48 INFO     [python.py:192] Python closure saved
19-10-22:11:07:48 INFO     [python.py:210] Using Python 3.7 base image
19-10-22:11:07:48 INFO     [clipper_admin.py:537] [default-cluster] Building model Docker image with model data from /var/folders/k0/gp0mgjf54ml_rr6bx2c_h7bw0000gn/T/tmpny6my18lclipper
19-10-22:11:07:49 INFO     [clipper_admin.py:542] [default-cluster] Step 1/3 : FROM clipper/python37-closure-container:develop
19-10-22:11:07:49 INFO     [clipper_admin.py:542] [default-cluster]  ---> 2660542c0759
19-10-22:11:07:49 INFO     [clipper_admin.py:542] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential &

19-10-17:11:02:43 INFO     [clipper_admin.py:303] [default-cluster] Model scikit-learn-model is now linked to application test-app-1


In [124]:
# Deploy PyTorch
#create_clipper_application(CLIPPER_APP_2_NAME)
pytorch_model_name = 'pytorch-alg-2'
# Deploy to production but do not serve
deploy_pytorch(clipper_connection, pytorch_model, name=pytorch_model_name, version=6)
# Link and serve !
clipper_connection.link_model_to_app(app_name=CLIPPER_APP_2_NAME, model_name=pytorch_model_name)

19-10-22:11:57:46 INFO     [deployer_utils.py:41] Saving function to /var/folders/k0/gp0mgjf54ml_rr6bx2c_h7bw0000gn/T/tmpdi2q9b79clipper
19-10-22:11:57:46 INFO     [deployer_utils.py:51] Serialized and supplied predict function
19-10-22:11:57:46 INFO     [pytorch.py:204] Torch model saved
19-10-22:11:57:46 INFO     [pytorch.py:222] Using Python 3.7 base image
19-10-22:11:57:46 INFO     [clipper_admin.py:537] [default-cluster] Building model Docker image with model data from /var/folders/k0/gp0mgjf54ml_rr6bx2c_h7bw0000gn/T/tmpdi2q9b79clipper
19-10-22:11:57:47 INFO     [clipper_admin.py:542] [default-cluster] Step 1/2 : FROM clipper/pytorch37-container:develop
19-10-22:11:57:47 INFO     [clipper_admin.py:542] [default-cluster]  ---> 62eb5cc2807e
19-10-22:11:57:47 INFO     [clipper_admin.py:542] [default-cluster] Step 2/2 : COPY /var/folders/k0/gp0mgjf54ml_rr6bx2c_h7bw0000gn/T/tmpdi2q9b79clipper /model/
19-10-22:11:57:47 INFO     [clipper_admin.py:542] [default-cluster]  ---> 5e0bc80e4cb6

ClipperException: Received error status code: 400 and message: The model with name 'pytorch-alg-2' is already linked to 'test-app-2'

In [93]:
# Test Scikit Learn
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://fabio.service.consul:9999/clipper/{app_name}/predict".format(app_name=CLIPPER_APP_1_NAME), headers=headers, data=json.dumps({"input": list(np.random.random(10))})).json()

{'query_id': 0,
 'output': -1.0,
 'default': True,
 'default_explanation': 'No connected models found for query'}

In [131]:
# Test PyTorch
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://fabio.service.consul:9999/clipper/{app_name}/predict".format(app_name=CLIPPER_APP_2_NAME), headers=headers, data=json.dumps({"input": list(np.random.random(10))})).json()

{'query_id': 10, 'output': [150.98452758789062], 'default': False}